In [1]:
!pip install pyspark==3.1.2 py4j==0.10.9

     |████████████████████████████████| 212.4 MB 58 kB/s 
     |████████████████████████████████| 198 kB 47.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=7810ab76e234a8a409eb36a3da7689898822393bd5ba7a6a43d2ff3d0f3fcfef
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.ml.stat import Correlation
from pyspark.sql.types import IntegerType,BooleanType,DateType,FloatType

spark = SparkSession.builder\
        .master("local[*]")\
        .appName('spotify_classification')\
        .getOrCreate()

In [3]:
from google.colab import drive               
drive.mount('/content/drive')

Mounted at /content/drive


In [39]:
df = spark.read.json("drive/My Drive/dataset.json")

In [40]:
df.show(5)

+------------+------------------+--------------------+---------------------+------------+-----------+------+--------------------+--------------------+----------------+---+--------+--------+----+----------------+------------+-----------+--------------------+---------------------+-------+--------------+-------+
|acousticness|               age|avg_artist_followers|avg_artist_popularity|danceability|duration_ms|energy|              genres|            id_track|instrumentalness|key|liveness|loudness|mode|popularity_track|release_date|speechiness|sum_artist_followers|sum_artist_popularity|  tempo|time_signature|valence|
+------------+------------------+--------------------+---------------------+------------+-----------+------+--------------------+--------------------+----------------+---+--------+--------+----+----------------+------------+-----------+--------------------+---------------------+-------+--------------+-------+
|       0.658| 41.83013698630137|              5403.5|             

In [41]:
from pyspark.ml.feature import QuantileDiscretizer 

qds = QuantileDiscretizer(relativeError=0.0001, handleInvalid="error", numBuckets=3, inputCol="popularity_track", outputCol="label_equi")

df = qds.setHandleInvalid("keep").fit(df).transform(df)

In [42]:
from pyspark.ml.feature import Bucketizer
bucketBorders=[0,33,66,100]
bucketer=Bucketizer().setSplits(bucketBorders).setInputCol("popularity_track").setOutputCol("label")
df = bucketer.transform(df)

In [43]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler( 
inputCols=[
 'duration_ms',
 'danceability',
 'energy',
 'key',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'time_signature',
 'sum_artist_followers',
 'sum_artist_popularity',
 'avg_artist_popularity'], 
outputCol="feat")
df=assembler.transform(df)

In [44]:
from pyspark.ml.feature import StandardScaler


scaler = StandardScaler(inputCol="feat", outputCol="features")


scalerModel = scaler.fit(df)


df = scalerModel.transform(df)

In [45]:
final_data = df.select("id_track", "features", "label_equi","label")

In [46]:
train, test = final_data.randomSplit([0.7, 0.3], seed = 10)

In [47]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Random forest

In [48]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(featuresCol="features",labelCol="label",maxDepth = 10)
rfModel = rf.fit(train)
predictions = rfModel.transform(test)

In [49]:
rfModel.featureImportances

SparseVector(15, {0: 0.0502, 1: 0.025, 2: 0.0493, 3: 0.0011, 4: 0.0005, 5: 0.0256, 6: 0.1732, 7: 0.0282, 8: 0.0093, 9: 0.0107, 10: 0.0032, 11: 0.0013, 12: 0.1193, 13: 0.1985, 14: 0.3048})

In [50]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="accuracy")
evaluator.evaluate(predictions)

0.7318884354982105

In [51]:
evaluator = MulticlassClassificationEvaluator(labelCol="label_equi", predictionCol="prediction",metricName="accuracy")
evaluator.evaluate(predictions)

0.4049114368431851

In [52]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="f1")
evaluator.evaluate(predictions)

0.7210829603782273

In [53]:
evaluator = MulticlassClassificationEvaluator(labelCol="label_equi", predictionCol="prediction",metricName="f1")
evaluator.evaluate(predictions)

0.30248261762036366

In [54]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="recallByLabel") 
evaluator.evaluate(predictions)

0.8308701072526385

In [55]:
evaluator = MulticlassClassificationEvaluator(labelCol="label_equi", predictionCol="prediction",metricName="recallByLabel") 
evaluator.evaluate(predictions)

0.9270631025106819

In [56]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="precisionByLabel") 
evaluator.evaluate(predictions)

0.7740462617361007

In [57]:
evaluator = MulticlassClassificationEvaluator(labelCol="label_equi", predictionCol="prediction",metricName="precisionByLabel") 
evaluator.evaluate(predictions)

0.465516477086344

# MLP

In [58]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

In [61]:
layers = [15,10,3,3]

In [70]:
mlp = MultilayerPerceptronClassifier(labelCol='label',
                                            featuresCol='features',
                                            maxIter=100,
                                            layers=layers,
                                            blockSize=128,
                                            seed=1234)

mlpModel = mlp.fit(train)


predictions = mlpModel.transform(test)

In [71]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="accuracy")
evaluator.evaluate(predictions)

0.724063646932774

In [72]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="f1")
evaluator.evaluate(predictions)

0.7139933174151885

In [73]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="recallByLabel") 
evaluator.evaluate(predictions)

0.817147098849437

In [74]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="precisionByLabel") 
evaluator.evaluate(predictions)

0.7716814954614901

In [75]:
mlp = MultilayerPerceptronClassifier(labelCol='label_equi',
                                            featuresCol='features',
                                            maxIter=100,
                                            layers=layers,
                                            blockSize=128,
                                            seed=1234)

mlpModel = mlp.fit(train)


predictions = mlpModel.transform(test)

In [76]:
evaluator = MulticlassClassificationEvaluator(labelCol="label_equi", predictionCol="prediction",metricName="accuracy")
evaluator.evaluate(predictions)

0.5807383682494814

In [77]:
evaluator = MulticlassClassificationEvaluator(labelCol="label_equi", predictionCol="prediction",metricName="f1")
evaluator.evaluate(predictions)

0.5791783677760735

In [78]:
evaluator = MulticlassClassificationEvaluator(labelCol="label_equi", predictionCol="prediction",metricName="recallByLabel") 
evaluator.evaluate(predictions)

0.659645467707193

In [79]:
evaluator = MulticlassClassificationEvaluator(labelCol="label_equi", predictionCol="prediction",metricName="precisionByLabel") 
evaluator.evaluate(predictions)

0.6554906398582382